In [ ]:
!git clone https://github.com/wilsonjr/cramming.git

In [ ]:
!pip install cramming/.

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash

In [ ]:
!pip install wandb

In [12]:
!git config --global user.email "wilsonjr@outlook.com"
!git config --global user.name "wilsonjr"

In [14]:
!git config --global credential.helper store

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import wandb
wandb.login()

In [ ]:
!python cramming/pretrain.py\
   name=cramming-bert-subset-pt\
   arch=crammed-bert\
   train=bert-o4\
   data=giga-verbo\
   wandb.enabled=True\
   wandb.project=cramming-bert\
   wandb.tags=[test,gigaverbo,subset,crammedBERT]\
   budget=1\
   impl.microbatch_size=64\
   train.batch_size=4096\
   impl.push_to_huggingface_hub=True\
   impl.hf_directoy_name=test-crammedBERT-subset-GigaVerbo

In [ ]:
# !python cramming/pretrain.py\
#     name=amp_b8192_cb_o4_final\
#     arch=crammed-bert\
#     train=bert-o4\
#     data=pile-readymade\
#     wandb.enabled=True\
#     wandb.project=cramming-bert\
#     wandb.tags=[test,pile,preprocessed,crammedBERT]\
#     budget=24\
#     impl.push_to_huggingface_hub=True\
#     impl.hf_directoy_name=bert-pile-crammed

In [ ]:
!python load_local_model.py\
    name=cramming-bert-subset-pt\
        impl.push_to_huggingface_hub=True\
        impl.hf_directoy_name=test-crammedBERT-subset-GigaVerbo\
        base_dir=/Users/wilson/Documents/projects/cramming/outputs/\
        +eval=GLUE_sane impl.compile_torch=False

In [ ]:
import cramming # needed to load crammed model
from transformers import AutoModelForMaskedLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("wilsonmarciliojr/test-crammedBERT-subset-GigaVerbo")
model  = AutoModelForMaskedLM.from_pretrained("wilsonmarciliojr/test-crammedBERT-subset-GigaVerbo")

text = "Oi, eu sou um modelo <mask>."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)


In [ ]:
import torch

mask_token_index = (encoded_input["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

mask_token_logits = output['outputs'][mask_token_index, :]

top_k = 5
top_k_values, top_k_indices = torch.topk(mask_token_logits, top_k, dim=1)

predicted_tokens = [tokenizer.decode([token_idx]) for token_idx in top_k_indices[0].tolist()]
print("Top-k predictions for the mask token:", predicted_tokens)

In [ ]:
# use output to get the predictions from the model
predictions = tokenizer.decode(output['outputs'].argmax(dim=-1))
predictions

# the topk predictions for <mask> token

In [ ]:
# Use a pipeline as a high-level helper
import cramming
from transformers import pipeline

pipe = pipeline("fill-mask", model="JonasGeiping/crammed-bert", tokenizer=tokenizer)  

In [ ]:
tokenizer.mask_token

In [ ]:
pipe("Hello, I'm a <mask> model.")